In [47]:
from tensorflow.keras.utils import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
#from sklearn.model_selection import train_test_split
import tensorflow as tf
import librosa
from tensorflow import keras
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
#import seaborn as sns
import numpy as np
from pydub import AudioSegment
from pydub.playback import play
from scipy.io import wavfile
from tempfile import mktemp

from datasets import load_dataset

In [158]:
TIME_STEPS = 2
image_name = 0
#TODO convert the processed data arrays to numpy
x_processed_data = []
y_processed_data = []

In [159]:
ds = load_dataset("mozilla-foundation/common_voice_11_0", "en", split='train[:100]')

Using the latest cached version of the module from /home/hetricke/.cache/huggingface/modules/datasets_modules/datasets/mozilla-foundation--common_voice_11_0/3f27acf10f303eac5b6fbbbe02495aeddb46ecffdb0a2fe3507fcfbf89094631 (last modified on Thu Sep  7 18:05:10 2023) since it couldn't be found locally at mozilla-foundation/common_voice_11_0., or remotely on the Hugging Face Hub.
Found cached dataset common_voice_11_0 (/home/hetricke/.cache/huggingface/datasets/mozilla-foundation___common_voice_11_0/en/11.0.0/3f27acf10f303eac5b6fbbbe02495aeddb46ecffdb0a2fe3507fcfbf89094631)


In [160]:
def detect_leading_silence(sound, silence_threshold=-50.0, chunk_size=10):
    '''
    sound is a pydub.AudioSegment
    silence_threshold in dB
    chunk_size in ms

    iterate over chunks until you find the first one with sound
    '''
    trim_ms = 0 # ms

    assert chunk_size > 0 # to avoid infinite loop
    while sound[trim_ms:trim_ms+chunk_size].dBFS < silence_threshold and trim_ms < len(sound):
        trim_ms += chunk_size

    return trim_ms

In [161]:
#TODO: convert saved transcript to numpy array
def transcript_prep(transcription, audio_len):
    if transcription.startswith('"') and transcription.endswith('"'):
        # we can remove trailing quotation marks as they do not affect the transcription
        transcription = transcription[1:-1]

    if len(transcription) > 0:
        if transcription[-1] not in [".", "?", "!"]:
            # append a full-stop to sentences that do not end in punctuation
            transcription = transcription + "."

        
        t_len = len(transcription)
        t_len_mod = len(transcription)%audio_len
        t_step = (t_len-t_len_mod)/audio_len

        transcript_pieces = []
        index=0

        while (index < t_len-t_len_mod-1):
            transcript_pieces.append(transcription[index:int(index+t_step)])
            index = int(index + t_step)

        transcript_pieces.append(transcript_pieces[index:t_len])

    return transcript_pieces


In [164]:
#TODO: convert saved data features to numpy array
def prepare_dataset(batch):

    global image_name
    global x_processed_data
    global y_processed_data

    #reads in mp3
    mp3_audio = AudioSegment.from_file(batch['path'], format="mp3")  # read mp3

    if(round(mp3_audio.duration_seconds) == 0):
        return batch

    #removes silent audio from the beginning and end
    start_trim = detect_leading_silence(mp3_audio)
    end_trim = detect_leading_silence(mp3_audio.reverse())
    duration = len(mp3_audio)    
    trimmed_sound = mp3_audio[start_trim:duration-end_trim]

    #converts the mp3 into a wav file
    wname = mktemp('.wav')  # use temporary file
    trimmed_sound.export(wname, format="wav")  # convert to wav
    FS, audio_data = wavfile.read(wname)  # read wav file


    #creates a file name for the spectrogram
    file_name = "images/"+ str(image_name) + ".png"
    image_name = image_name + 1

    #creates and saves the spectrogram
    plt.figure()
    plt.specgram(audio_data, Fs=FS, NFFT=128, noverlap=0)  # plot
    plt.axis('off')
    plt.savefig(file_name, bbox_inches='tight')

    #clears the figure for the next audio transcript- otherwise it just overwrites the image, which causes Problems
    plt.close()


    #loads the spectrogram and turns it into an array
    img = keras.preprocessing.image.load_img(file_name)
    img_array = keras.preprocessing.image.img_to_array(img)


    #slices the image array into appriopriate sizes for each chunk
    samples = round(mp3_audio.duration_seconds)
    features = int(img_array.shape[0]/samples)

    divided_img_array = []

    index = 0

    for i in range(samples-1):
        feature_array = img_array[index:index+features]
        new_dim = feature_array.shape[0]*feature_array.shape[1]
        feature_array = feature_array.reshape(new_dim, -1)

        original_length = feature_array.shape[0]
        padding = 66908-original_length
        feature_array = np.pad(feature_array, [(0,padding), (0,0)], mode='constant')

        divided_img_array.append(feature_array)
        
        index = index + features

    x_data = []

    for i in range(samples-TIME_STEPS):
        x_data.append([])
        for j in range(TIME_STEPS):
            x_data[i].append(divided_img_array[j])



    x_processed_data.append(x_data)
    y_processed_data = transcript_prep(batch["sentence"], samples)
    


    #reshape the spectogram into a 3d array that goes (num samples x timesteps x features)
    #1 sample is how many times this gets run through
    #timesteps is how far backward to grab (so that's a value that can be played with!)
    #num samples is I think how long it is (so grab the mp3 length)
    #slice the image accordingly
    #convert the image into an array (reference the obj detection) to use for the features
    #profit

    #so the min time is three, which, at 515 timesteps, gets 171.6666 columns of data per second

    return batch


In [165]:
image_name = 0
ds = ds.map(prepare_dataset, desc="preprocess dataset")

preprocess dataset:  29%|██▉       | 29/100 [00:05<00:13,  5.26 examples/s]/home/hetricke/anaconda3/envs/speech_recog_env/lib/python3.11/site-packages/matplotlib/axes/_axes.py:7939: RuntimeWarning: divide by zero encountered in log10
  Z = 10. * np.log10(spec)
preprocess dataset:  64%|██████▍   | 64/100 [00:12<00:07,  4.84 examples/s]/home/hetricke/anaconda3/envs/speech_recog_env/lib/python3.11/site-packages/matplotlib/axes/_axes.py:7939: RuntimeWarning: divide by zero encountered in log10
  Z = 10. * np.log10(spec)
preprocess dataset:  83%|████████▎ | 83/100 [00:16<00:03,  5.01 examples/s]/home/hetricke/anaconda3/envs/speech_recog_env/lib/python3.11/site-packages/matplotlib/axes/_axes.py:7939: RuntimeWarning: divide by zero encountered in log10
  Z = 10. * np.log10(spec)
preprocess dataset:  85%|████████▌ | 85/100 [00:17<00:03,  4.84 examples/s]/home/hetricke/anaconda3/envs/speech_recog_env/lib/python3.11/site-packages/matplotlib/axes/_axes.py:7939: RuntimeWarning: divide by zero enco

In [172]:
# split = ds.train_test_split(test_size = 0.2,train_size = 0.8)
# train = split['train']
# test = split['test']
x_processed_data = np.array(x_processed_data)
y_processed_data = np.array(y_processed_data)

x_train = x_processed_data[:80]
y_train = y_processed_data[:80]

x_test = x_processed_data[:-20]
y_test = y_processed_data[:-20]


/tmp/ipykernel_31204/3975215461.py:4: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  x_processed_data = np.array(x_processed_data)
/tmp/ipykernel_31204/3975215461.py:5: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  y_processed_data = np.array(y_processed_data)


In [167]:
#build model
model = keras.Sequential()
model.add(keras.layers.SimpleRNN(units = 128, activation = 'relu', input_shape = (66908, 3)))
model.add(keras.layers.Dense(units = 128, activation = 'relu'))
model.add(keras.layers.Dense(units = 128, activation='relu'))
model.add(keras.layers.Dense(units = 128, activation ='relu'))
model.add(keras.layers.Dense(29, activation = 'softmax'))
model.compile(optimizer='adam', metrics=['accuracy'])

In [171]:
print(x_train[0])
print(y_train[0])

KeyboardInterrupt: 

In [173]:
#train model
#what you might have to do is iteratively run fit on all the audio samples

model.fit(x_train[0], y_train[0], epochs=10, batch_size=1)


ValueError: Failed to find data adapter that can handle input: (<class 'list'> containing values of types {'(<class \'list\'> containing values of types {"<class \'numpy.ndarray\'>"})'}), <class 'str'>

In [ ]:
#add computer i/o for speech recognition